In [11]:
import geopandas as gpd
import pandas as pd
import os

In [3]:
path = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources\Network_Analysis\Network_Analysis.gdb"

In [26]:
baseyear_jobs_pts = gpd.read_file(path, layer="baseyearJobs_FeatureToPoint")

In [10]:
baseyear_jobs_pts.columns

Index(['btype', 'nrsqft', 'rsqft', 'du', 'yrbuilt', 'lpid', 'pundev',
       'dev_land', 'developed', 'obtype', 'orsqft', 'onrsqft', 'odu', 'zid',
       'rezoned', 'city', 'annexed', 'ozid', 'AreaPerJob', 'isNonRes', 'jobs',
       'ojobs', 'hh', 'ohh', 'ORIG_FID', 'geometry'],
      dtype='object')

In [5]:
baseyear_jobs_pts.head()

,btype,nrsqft,rsqft,du,yrbuilt,lpid,pundev,dev_land,developed,obtype,...,annexed,ozid,AreaPerJob,isNonRes,jobs,ojobs,hh,ohh,ORIG_FID,geometry
0,"(5100,)",1662.0,0.0,0.0,1969.0,24356.0,0.15,1.973955e+06,0.0,"(5100,)",...,0.0,119.0,720.152698,1,2.307844,2.307844,0.0,0.0,0,POINT (4253553.765 844474.220)
1,"(5100,)",1840.0,0.0,0.0,1988.0,25261.0,0.00,1.103922e+05,0.0,"(5100,)",...,0.0,126.0,720.152698,1,2.555014,2.555014,0.0,0.0,1,POINT (4260623.851 852946.585)
2,"(5100,)",4440.0,0.0,0.0,1964.0,25322.0,0.00,1.150251e+05,0.0,"(5100,)",...,0.0,58.0,720.152698,1,6.165359,6.165359,0.0,0.0,2,POINT (4271080.535 853188.657)
3,"(5100,)",8000.0,0.0,0.0,1999.0,25351.0,0.00,2.108553e+05,0.0,"(5100,)",...,0.0,58.0,720.152698,1,11.108755,11.108755,0.0,0.0,3,POINT (4260673.648 853441.487)
4,"(5200,)",14375.0,0.0,0.0,1990.0,25393.0,0.00,2.173801e+05,0.0,"(5200,)",...,0.0,125.0,624.844490,1,23.005724,23.005724,0.0,0.0,4,POINT (4260190.606 853680.107)


In [6]:
input_folder = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources"

In [8]:
service_stops = gpd.read_file(os.path.join(input_folder, "PerformanceAnalysis", "service_transit_equity", "service_stops.shp"))

In [9]:
service_stops.head()

,name,longitude,latitude,type,geometry
0,Coburg City Hall,-123.067629,44.139154,City Hall,POINT (-13699827.424 5487002.751)
1,LTD Park & Ride St. Matthew?s Episcopal Church,-123.140940,44.122960,LTD Park & Ride,POINT (-13707988.283 5484491.106)
2,Awbrey Park Elementary School,-123.137966,44.123737,Elementary Sch,POINT (-13707657.245 5484611.544)
3,Madison Middle School,-123.119675,44.117052,Middle School,POINT (-13705621.082 5483574.996)
4,Spring Creek Elementary School,-123.144327,44.114781,Elementary Sch,POINT (-13708365.315 5483222.802)


In [12]:
building_types = pd.read_csv('building_types.csv')

In [14]:
btypes = building_types[building_types['is_non_residential']]['building_type_id'].unique()

In [16]:
building_types.head()

,building_type_id,name,description,is_residential,is_non_residential,is_other
0,1110,Single-family Detached,Single Family Detached,True,False,False
1,1121,Townhome or Rowhouse,"Single-Family Attached, Townhome or Rowhouse",True,False,False
2,1122,Duplex,"Single-Family Attached, Duplex",True,False,False
3,1130,Mobile Homes,"Single Family, Mobile Home",True,False,False
4,1210,Multifamily - Small Building,"Multifamily, Triplex or Small (3 to 19 unit) A...",True,False,False


In [17]:
def get_building_info(x):
    if len(x[1:-2].split(', ')) == 1:
        a = x[1:-2].split(', ')
        bname = building_types.loc[building_types.building_type_id == int(a[0]), 'name'].values[0]
        bdesc = building_types.loc[building_types.building_type_id == int(a[0]), 'description'].values[0]
    else:
        a = x[1:-1].split(', ')
        b = [int(btype) for btype in a]
        btypeList = [btype for btype in b if btype in btypes]
        bname = building_types.loc[building_types.building_type_id.isin(btypeList), 'name'].values[0]
        bdesc = building_types.loc[building_types.building_type_id.isin(btypeList), 'description'].values[0]
    return(bname, bdesc)

In [27]:
baseyear_jobs_pts['type'] = baseyear_jobs_pts['btype'].apply(lambda x: get_building_info(x)[0])

In [28]:
baseyear_jobs_pts['description'] = baseyear_jobs_pts['btype'].apply(lambda x: get_building_info(x)[1])

In [29]:
service_stops = service_stops.rename(columns={"name":"description"})

In [30]:
cols = ['type', 'description', 'geometry']

In [31]:
service_stops = service_stops[cols]

In [32]:
baseyear_jobs_pts = baseyear_jobs_pts[cols]

In [33]:
baseyear_amenities = baseyear_jobs_pts.append(service_stops)

In [34]:
baseyear_amenities.head()

,type,description,geometry
0,Light Industrial / Warehousing,"Industrial, Light / Warehousing",POINT (4253553.765 844474.220)
1,Light Industrial / Warehousing,"Industrial, Light / Warehousing",POINT (4260623.851 852946.585)
2,Light Industrial / Warehousing,"Industrial, Light / Warehousing",POINT (4271080.535 853188.657)
3,Light Industrial / Warehousing,"Industrial, Light / Warehousing",POINT (4260673.648 853441.487)
4,Heavy Industrial,"Industrial, Heavy",POINT (4260190.606 853680.107)


In [35]:
forecast_jobs_pts = gpd.read_file(path, layer="forecastJobs_FeatureToPoint")

In [36]:
forecast_jobs_pts['type'] = forecast_jobs_pts['btype'].apply(lambda x: get_building_info(x)[0])
forecast_jobs_pts['description'] = forecast_jobs_pts['btype'].apply(lambda x: get_building_info(x)[1])

In [37]:
forecast_jobs_pts = forecast_jobs_pts[cols]

In [38]:
forecast_amenities = forecast_jobs_pts.append(service_stops)

In [40]:
out_folder = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources"

In [41]:
baseyear_amenities.to_file(os.path.join(out_folder, 'baseyear_amenities.shp'))

In [42]:
forecast_amenities.to_file(os.path.join(out_folder, 'forecast_amenities.shp'))